In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("/kaggle/input/model-info/model_info_2.csv")

In [7]:
df.head()

,model_names,model_descriptions,model_descriptions_structured,model_parameters,model_summary
0,DynamicConvNet,DynamicConvNet(\n (conv_layers): Sequential(\...,Layer1(Type=Conv2d | In_channels=3 | Out_chann...,61618,This model is a Dynamic Convolutional Neural N...
1,DynamicConvNet,DynamicConvNet(\n (conv_layers): Sequential(\...,Layer1(Type=Conv2d | In_channels=3 | Out_chann...,31582,This model is a DynamicConvNet consisting of a...
2,DynamicConvNet,DynamicConvNet(\n (conv_layers): Sequential(\...,Layer1(Type=Conv2d | In_channels=3 | Out_chann...,48904,This Dynamic Convolutional Neural Network (Dyn...
3,DynamicConvNet,DynamicConvNet(\n (conv_layers): Sequential(\...,Layer1(Type=Conv2d | In_channels=3 | Out_chann...,22144,The DynamicConvNet is a Convolutional Neural N...
4,DynamicConvNet,DynamicConvNet(\n (conv_layers): Sequential(\...,Layer1(Type=Conv2d | In_channels=3 | Out_chann...,32494,The model is a Dynamic Convolutional Neural Ne...


In [ ]:
for i in range(len(df)):
    df["model_descriptions_structured"][i] = df["model_descriptions_structured"][i].replace("\n", ",")

In [9]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [10]:
train_df=df
# train_df=train_df.sample(frac = 1)
batch_size=1
num_of_batches=len(train_df)/batch_size

In [11]:
dev = torch.device("cuda")

In [12]:
tokenizer = T5Tokenizer.from_pretrained("google/t5-efficient-tiny")
model = T5ForConditionalGeneration.from_pretrained("google/t5-efficient-tiny",
                                             return_dict=True)
# moving the model to GPU
model.to("cuda")

tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/62.3M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 256)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 256)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=256, out_features=256, bias=False)
              (k): Linear(in_features=256, out_features=256, bias=False)
              (v): Linear(in_features=256, out_features=256, bias=False)
              (o): Linear(in_features=256, out_features=256, bias=False)
              (relative_attention_bias): Embedding(32, 4)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=256, out_features=1024, bias=False)
              (wo): Linear(in_features=1024, out_features=256, bias=False)
              (dropout): Drop

In [13]:
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [14]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress    
      value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [16]:
#Sets the module in training mode
model.train()
num_of_epochs = 100
loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
  
    running_loss=0

    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(int(num_of_batches)):
        inputbatch=[]
        labelbatch=[]
        new_df=train_df[i*batch_size:i*batch_size+batch_size]
        for indx, row in new_df.iterrows():
            input = 'LNE:'+row['model_descriptions_structured']+ " | Model Parameters: " + str(row['model_parameters'])+'</s>' 
            labels = row['model_summary']+'</s>'   
            inputbatch.append(input)
            labelbatch.append(labels)
        inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=600,return_tensors='pt')["input_ids"]
        labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
        # clear out the gradients of all Variables 
        optimizer.zero_grad()

        # Forward propogation
        outputs = model(input_ids=inputbatch.to(dev), labels=labelbatch.to(dev))
        loss = outputs.loss
        del(outputs)
        loss_num=loss.item()
        running_loss+=loss_num
        if i%10 ==0:      
            loss_per_10_steps.append(loss_num)
        out.update(progress(loss_num,i, num_of_batches+1))

        # calculating the gradients
        loss.backward()

        #updating the params
        optimizer.step()
    
    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))

Running epoch: 1


Epoch: 1 , Running loss: 2.371063953294204
Running epoch: 2


Epoch: 2 , Running loss: 1.8078628400197396
Running epoch: 3


Epoch: 3 , Running loss: 1.6069806163700728
Running epoch: 4


Epoch: 4 , Running loss: 1.4781549281512316
Running epoch: 5


...

Epoch: 96 , Running loss: 0.3284391163919981
Running epoch: 97


Epoch: 97 , Running loss: 0.323360899484788
Running epoch: 98


Epoch: 98 , Running loss: 0.32067556616563636
Running epoch: 99


Epoch: 99 , Running loss: 0.31613567240822776
Running epoch: 100


Epoch: 100 , Running loss: 0.3154132157205962


In [20]:
df["model_descriptions"][2]

'DynamicConvNet(\n  (conv_layers): Sequential(\n    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))\n    (1): ReLU(inplace=True)\n    (2): Dropout(p=0.5, inplace=False)\n    (3): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))\n    (4): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n    (5): ReLU(inplace=True)\n    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)\n    (7): Conv2d(12, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))\n    (8): ReLU(inplace=True)\n    (9): Dropout(p=0.5, inplace=False)\n  )\n  (fc): Linear(in_features=4608, out_features=10, bias=True)\n)\n'

In [22]:
structured_data = df["model_descriptions_structured"][2]
parameters  = df["model_parameters"][2]
prompt = f"{structured_data}. The model parameters is {parameters}"

In [23]:
prompt

'Layer1(Type=Conv2d | In_channels=3 | Out_channels=6 | Kernel_size=(3,3) | Stride=(1,1) | Padding=(1,1)),Layer2(Type=Conv2d | In_channels=6 | Out_channels=12 | Kernel_size=(3,3) | Stride=(1,1) | Padding=(1,1)),Layer3(Type=Conv2d | In_channels=12 | Out_channels=18 | Kernel_size=(3,3) | Stride=(1,1) | Padding=(1,1)),Layer4(Type=ReLU | Inplace=True),Layer5(Type=ReLU | Inplace=True),Layer6(Type=ReLU | Inplace=True),Layer7(Type=Dropout | Probability=0.5 | Inplace=False),Layer8(Type=Dropout | Probability=0.5 | Inplace=False),Layer9(Type=MaxPool2d | Kernel_size=2 | Stride=2 | Padding=0 | Dilation=1 | Ceil_mode=False),Layer10(Type=Linear | In_features=4608 | Out_features=10 | Bias=True). The model parameters is 48904'

In [29]:
def generate(text):
   model.eval()
   input_ids = tokenizer.encode("LNE:{} </s>".format(text), 
                               return_tensors="pt")  
   outputs = model.generate(input_ids.cuda())
   return tokenizer.decode(outputs[0])

In [32]:
generate(prompt)

'The model is a deep neural network with a total of 10 layers, including convolutional layers, ReLU activation layers, and dropout layers. The input to the model is an image with 3 channels (RGB) and a size of 32x32 pixels and the output of the model is a vector of length 10, which represents the predicted probabilities for each of the 10 possible classes. The model has a total of 48,904 trainable parameters.'